### INFO 3401

Week 5 relational data: applications

In [2]:
import numpy as np
import pandas as pd
import sqlite3
import os

- DISTINCT
- COUNT
- WHERE 
- ORDER BY
- MIN/MAX

Downlod the [Lahman](https://www.kaggle.com/datasets?fileType=sqlite&sizeEnd=50%2CMB) database from Kaggle.


In [10]:
# here is how you connect to the Lahman database and describe its tables

import sqlite3

con = sqlite3.connect("lahmansbaseballdb.sqlite")

# get the db name
db_name = pd.read_sql("PRAGMA database_list;", con)["name"][0]

In [11]:
# the name of main table in your database might be different
lahmans = pd.read_sql("SELECT * FROM {}.sqlite_master WHERE type='table';".format(db_name), con=con)

lahmans

,type,name,tbl_name,rootpage,sql
0,table,all_star,all_star,2,"CREATE TABLE all_star (\n player_id TEXT,\n..."
1,table,appearances,appearances,217,"CREATE TABLE appearances (\n year INTEGER,\..."
2,table,manager_award,manager_award,5278,CREATE TABLE manager_award (\n player_id TE...
3,table,player_award,player_award,5288,CREATE TABLE player_award (\n player_id TEX...
4,table,manager_award_vote,manager_award_vote,5560,CREATE TABLE manager_award_vote (\n award_i...
5,table,player_award_vote,player_award_vote,5580,CREATE TABLE player_award_vote (\n award_id...
6,table,batting,batting,5817,"CREATE TABLE batting (\n player_id TEXT,\n ..."
7,table,batting_postseason,batting_postseason,11278,CREATE TABLE batting_postseason (\n year IN...
8,table,player_college,player_college,11882,CREATE TABLE player_college (\n player_id T...
9,table,fielding,fielding,12362,"CREATE TABLE fielding (\n player_id TEXT,\n..."


### What has happened so far ?

1. We downloaded a database from Kaggle
2. We loaded it into the notebook
3. We found out the database name and described the tables

I just printed out the structure of the database and am curious about what is happening. Let's take a look at the hall of fame table.

In [32]:
pd.read_sql_query('select * from hall_of_fame limit 5',con)

,player_id,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
0,cobbty01,1936,BBWAA,226,170,222,Y,Player,
1,ruthba01,1936,BBWAA,226,170,215,Y,Player,
2,wagneho01,1936,BBWAA,226,170,215,Y,Player,
3,mathech01,1936,BBWAA,226,170,205,Y,Player,
4,johnswa01,1936,BBWAA,226,170,189,Y,Player,


It looks like to find anything interesting from this table I need the players table.

In [33]:
pd.read_sql_query('select * from player limit 5', con)

,player_id,birth_year,birth_month,birth_day,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,name_last,name_given,weight,height,bats,throws,debut,final_game,retro_id,bbref_id
0,aardsda01,1981,12,27,USA,CO,Denver,,,,...,Aardsma,David Allan,220,75,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934,2,5,USA,AL,Mobile,,,,...,Aaron,Henry Louis,180,72,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,...,Aaron,Tommie Lee,190,75,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954,9,8,USA,CA,Orange,,,,...,Aase,Donald William,190,75,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972,8,25,USA,FL,Palm Beach,,,,...,Abad,Fausto Andres,184,73,L,L,2001-09-10,2006-04-13,abada001,abadan01


Looking again at the tables in the notebook I see there is also an all star table. Let's look at that. 

In [49]:
pd.read_sql_query('select * from all_star where player_id = "walkeje01"', con)

,player_id,year,game_num,game_id,team_id,league_id,gp,starting_pos
0,walkeje01,1959,2,NLS195908030,BAL,AL,1,1


Now I am starting to think of an interesting question. Are there players that were often all stars but who were not elected to the hall of fame? These are **forgotten stars**. They were robbed! To find this out, I need to find out how many all star games each player played in.

It seems like the all_star table has a game_num field. What does that mean? I have to check docs

In [50]:
pd.read_sql_query('select * from all_star where player_id = "walkeje01"', con)

,player_id,year,game_num,game_id,team_id,league_id,gp,starting_pos
0,walkeje01,1959,2,NLS195908030,BAL,AL,1,1


It seems like the game_num field says the all start game number the player played in, assuming there are more than 1 in a season. I can double check that by seeing all of the possible values of game num with the following query. Looking, it seems like there is no more than 2 for any player. 

In [52]:
pd.read_sql_query('select distinct game_num from all_star', con)

,game_num
0,0
1,2
2,1


In [104]:
all_stars = pd.read_sql_query('select all_star.player_id  from all_star inner join player on all_star.player_id = player.player_id where gp=1', con)

In [105]:
all_star_counts = all_stars["player_id"].value_counts().to_frame().reset_index()

In [118]:
all_star_counts = all_star_counts.rename(columns={"player_id": "N_all_star_games", "index":"player_id"})

In [119]:
pd.read_sql_query('select * from player where player_id = "aaronha01"', con)

,player_id,birth_year,birth_month,birth_day,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,name_last,name_given,weight,height,bats,throws,debut,final_game,retro_id,bbref_id
0,aaronha01,1934,2,5,USA,AL,Mobile,,,,...,Aaron,Henry Louis,180,72,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01


Henry Louis Allen played in 25 all star games. Is that right? https://en.wikipedia.org/wiki/Hank_Aaron#:~:text=Aaron%20was%20an%20NL%20All,Glove%20winner%20for%20three%20seasons.

Actually I see from this wikipedia page that Aaron was credited with 24 games. So I amended my SQL query about to set gp=1. This is an important validation when doing data analysis.

In [140]:
hall = pd.read_sql_query('select  * from hall_of_fame', con)

merged = pd.merge(all_star_counts, hall, how='left', on='player_id')

0       Y
1       Y
2       Y
3       Y
4       Y
       ..
3197    N
3198    N
3199    N
3200    N
3201    N
Name: inducted, Length: 3202, dtype: object

In [144]:
reduced = merged[['player_id', "N_all_star_games", "inducted"]]
reduced.fillna(value="N")

,player_id,N_all_star_games,inducted
0,musiast01,24,Y
1,aaronha01,24,Y
2,mayswi01,24,Y
3,ripkeca01,18,Y
4,robinbr01,18,Y
...,...,...,...
3197,willido03,1,N
3198,nevinph01,1,N
3199,whitter01,1,N
3200,youngma01,1,N


In [153]:
most_robbed = reduced[reduced["inducted"] == "N"].drop_duplicates().head(10)

In [154]:
players = pd.read_sql_query('select * from player', con)

merged = pd.merge(most_robbed, players, how='inner', on='player_id')

In [161]:
merged

,player_id,N_all_star_games,inducted,birth_year,birth_month,birth_day,birth_country,birth_state,birth_city,death_year,...,name_last,name_given,weight,height,bats,throws,debut,final_game,retro_id,bbref_id
0,rosepe01,16,N,1941,4,14,USA,OH,Cincinnati,,...,Rose,Peter Edward,192,71,B,R,1963-04-08,1986-08-17,rosep001,rosepe01
1,berrayo01,15,N,1925,5,12,USA,MO,St. Louis,2015,...,Berra,Lawrence Peter,185,67,L,R,1946-09-22,1965-05-09,berry101,berrayo01
2,foxne01,13,N,1927,12,25,USA,PA,St. Thomas,1975,...,Fox,Jacob Nelson,160,70,L,R,1947-06-08,1965-07-25,fox-n101,foxne01
3,bondsba01,13,N,1964,7,24,USA,CA,Riverside,,...,Bonds,Barry Lamar,185,73,L,L,1986-05-30,2007-09-26,bondb001,bondsba01
4,alomaro01,12,N,1968,2,5,P.R.,,Ponce,,...,Alomar,Roberto,184,72,B,R,1988-04-22,2004-09-05,alomr001,alomaro01
5,ottme01,11,N,1909,3,2,USA,LA,Gretna,1958,...,Ott,Melvin Thomas,170,69,L,R,1926-04-27,1947-07-11,ott-m101,ottme01
6,dimagjo01,11,N,1914,11,25,USA,CA,Martinez,1999,...,DiMaggio,Joseph Paul,193,74,R,R,1936-05-03,1951-09-30,dimaj101,dimagjo01
7,killeha01,11,N,1936,6,29,USA,ID,Payette,2011,...,Killebrew,Harmon Clayton,195,72,R,R,1954-06-23,1975-09-26,killh102,killeha01
8,piazzmi01,11,N,1968,9,4,USA,PA,Norristown,,...,Piazza,Michael Joseph,200,75,R,R,1992-09-01,2007-09-30,piazm001,piazzmi01
9,garvest01,10,N,1948,12,22,USA,FL,Tampa,,...,Garvey,Steven Patrick,192,70,R,R,1969-09-01,1987-05-23,garvs001,garvest01


In [159]:
hall = pd.read_sql_query('select  * from hall_of_fame where player_id="dimagjo01"', con)
hall

,player_id,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
0,dimagjo01,1945,BBWAA,247,186,1,N,Player,
1,dimagjo01,1953,BBWAA,264,198,117,N,Player,
2,dimagjo01,1954,BBWAA,252,189,175,N,Player,
3,dimagjo01,1955,BBWAA,251,189,223,Y,Player,


It looks like Joe DiMaggio was only [elected to the hall of fame](https://baseballhall.org/hall-of-famers/dimaggio-joe) on his fourth try. It also looks like Mel Ott was elected to the hall of fame. I have a mistake! To clean up, I am going to need to requery the hall_of_fame table setting inducted = Y

In [191]:
hall = pd.read_sql_query('select  * from hall_of_fame where inducted="Y"', con)
merged = pd.merge(all_star_counts, hall, how='left', on='player_id')
reduced = merged[['player_id', "N_all_star_games", "inducted"]]
reduced = reduced.fillna("N")
most_robbed = reduced[reduced["inducted"] == "N"].drop_duplicates().head(10)
players = pd.read_sql_query('select * from player', con)
merged = pd.merge(most_robbed, players, how='inner', on='player_id')
merged

,player_id,N_all_star_games,inducted,birth_year,birth_month,birth_day,birth_country,birth_state,birth_city,death_year,...,name_last,name_given,weight,height,bats,throws,debut,final_game,retro_id,bbref_id
0,rosepe01,16,N,1941,4,14,USA,OH,Cincinnati,,...,Rose,Peter Edward,192,71,B,R,1963-04-08,1986-08-17,rosep001,rosepe01
1,rodriiv01,14,N,1971,11,27,P.R.,,Manati,,...,Rodriguez,Ivan,205,69,R,R,1991-06-20,2011-09-28,rodri001,rodriiv01
2,bondsba01,13,N,1964,7,24,USA,CA,Riverside,,...,Bonds,Barry Lamar,185,73,L,L,1986-05-30,2007-09-26,bondb001,bondsba01
3,jeterde01,13,N,1974,6,26,USA,NJ,Pequannock,,...,Jeter,Derek Sanderson,195,75,R,R,1995-05-29,2014-09-28,jeted001,jeterde01
4,rodrial01,11,N,1975,7,27,USA,NY,New York,,...,Rodriguez,Alexander Enmanuel,225,75,R,R,1994-07-08,2015-10-04,rodra001,rodrial01
5,garvest01,10,N,1948,12,22,USA,FL,Tampa,,...,Garvey,Steven Patrick,192,70,R,R,1969-09-01,1987-05-23,garvs001,garvest01
6,boyerke01,10,N,1931,5,20,USA,MO,Liberty,1982,...,Boyer,Kenton Lloyd,190,73,R,R,1955-04-12,1969-08-09,boyek101,boyerke01
7,clemero02,10,N,1962,8,4,USA,OH,Dayton,,...,Clemens,William Roger,205,76,R,R,1984-05-15,2007-09-16,clemr001,clemero02
8,suzukic01,10,N,1973,10,22,Japan,Aichi,Nichi Kasugai-gun,,...,Suzuki,Ichiro,170,71,L,R,2001-04-02,2015-10-04,suzui001,suzukic01
9,riverma01,9,N,1969,11,29,Panama,Panama,Panama,,...,Rivera,Mariano,195,74,R,R,1995-05-23,2013-09-26,rivem002,riverma01
